In [64]:
import tensorflow.compat.v1 as tf


import numpy as np
import IPython.display as display

In [65]:
# with tf.Example.

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [66]:

print(_bytes_feature(b'test_string'))
print(_bytes_feature(u'test_bytes'.encode('utf-8')))

print(_float_feature(np.exp(1)))

print(_int64_feature(True))
print(_int64_feature(1))

bytes_list {
  value: "test_string"
}

bytes_list {
  value: "test_bytes"
}

float_list {
  value: 2.7182817459106445
}

int64_list {
  value: 1
}

int64_list {
  value: 1
}



In [67]:

feature = _float_feature(np.exp(1))

feature.SerializeToString()

b'\x12\x06\n\x04T\xf8-@'

In [68]:

# the number of observations in the dataset
n_observations = int(1e4)

# boolean feature, encoded as False or True
feature0 = np.random.choice([False, True], n_observations)

# integer feature, random from 0 .. 4
feature1 = np.random.randint(0, 5, n_observations)

# string feature
strings = np.array([b'cat', b'dog', b'chicken', b'horse', b'goat'])
feature2 = strings[feature1]

# float feature, from a standard normal distribution
feature3 = np.random.randn(n_observations)

In [69]:
def serialize_example(feature0, feature1, feature2, feature3):
  """
  Creates a tf.Example message ready to be written to a file.
  """

  # Create a dictionary mapping the feature name to the tf.Example-compatible
  # data type.

  feature = {
      'feature0': _int64_feature(feature0),
      'feature1': _int64_feature(feature1),
      'feature2': _bytes_feature(feature2),
      'feature3': _float_feature(feature3),
  }

  # Create a Features message using tf.train.Example.

  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

In [70]:
# This is an example observation from the dataset.

example_observation = []

serialized_example = serialize_example(False, 4, b'goat', 0.9876)
serialized_example

b'\nR\n\x11\n\x08feature0\x12\x05\x1a\x03\n\x01\x00\n\x11\n\x08feature1\x12\x05\x1a\x03\n\x01\x04\n\x14\n\x08feature2\x12\x08\n\x06\n\x04goat\n\x14\n\x08feature3\x12\x08\x12\x06\n\x04[\xd3|?'

In [71]:
example_proto = tf.train.Example.FromString(serialized_example)
example_proto

features {
  feature {
    key: "feature0"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "feature1"
    value {
      int64_list {
        value: 4
      }
    }
  }
  feature {
    key: "feature2"
    value {
      bytes_list {
        value: "goat"
      }
    }
  }
  feature {
    key: "feature3"
    value {
      float_list {
        value: 0.9876000285148621
      }
    }
  }
}

In [72]:

tf.data.Dataset.from_tensor_slices(feature1)

<DatasetV1Adapter shapes: (), types: tf.int64>

In [73]:
features_dataset = tf.data.Dataset.from_tensor_slices((feature0, feature1, feature2, feature3))
features_dataset

<DatasetV1Adapter shapes: ((), (), (), ()), types: (tf.bool, tf.int64, tf.string, tf.float64)>

In [74]:

# Use `take(1)` to only pull one example from the dataset.
for f0,f1,f2,f3 in features_dataset.batch(5).take(1):
  print(f0)
  print(f1)
  print(f2)
  print(f3)
  break

Tensor("IteratorGetNext_418:0", shape=(?,), dtype=bool)
Tensor("IteratorGetNext_418:1", shape=(?,), dtype=int64)
Tensor("IteratorGetNext_418:2", shape=(?,), dtype=string)
Tensor("IteratorGetNext_418:3", shape=(?,), dtype=float64)


In [75]:
(f0,f1,f2,f3) = next(iter(features_dataset))
f0.numpy()

AttributeError: 'Tensor' object has no attribute 'numpy'

In [76]:
with tf.Session() as sess:
    for f0,f1,f2,f3 in features_dataset.batch(5).take(1):
        print(f0)
        print(f1)
        print(f2)
        print(f3)
        sess.run([f0])
        break

Tensor("IteratorGetNext_420:0", shape=(?,), dtype=bool)
Tensor("IteratorGetNext_420:1", shape=(?,), dtype=int64)
Tensor("IteratorGetNext_420:2", shape=(?,), dtype=string)
Tensor("IteratorGetNext_420:3", shape=(?,), dtype=float64)


NotFoundError: Resource AnonymousIterator/AnonymousIterator9/N10tensorflow4data16IteratorResourceE does not exist.
	 [[node IteratorGetNext_420 (defined at <ipython-input-76-aa945dce992f>:2) ]]

Original stack trace for 'IteratorGetNext_420':
  File "/Users/songfeng/APP/env/python/anaconda3/envs/tensorflow14/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/songfeng/APP/env/python/anaconda3/envs/tensorflow14/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/songfeng/APP/env/python/anaconda3/envs/tensorflow14/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/songfeng/APP/env/python/anaconda3/envs/tensorflow14/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/Users/songfeng/APP/env/python/anaconda3/envs/tensorflow14/lib/python3.7/site-packages/ipykernel/kernelapp.py", 

In [77]:
a= tf.constant([1,2,34])

In [78]:
with tf.Session() as sess:
    _a = sess.run([f0])
    print(_a)

NotFoundError: Resource AnonymousIterator/AnonymousIterator10/N10tensorflow4data16IteratorResourceE does not exist.
	 [[node IteratorGetNext_420 (defined at <ipython-input-76-aa945dce992f>:2) ]]

Original stack trace for 'IteratorGetNext_420':
  File "/Users/songfeng/APP/env/python/anaconda3/envs/tensorflow14/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/songfeng/APP/env/python/anaconda3/envs/tensorflow14/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/songfeng/APP/env/python/anaconda3/envs/tensorflow14/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/songfeng/APP/env/python/anaconda3/envs/tensorflow14/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/Users/songfeng/APP/env/python/anaconda3/envs/tensorflow14/lib/python3.7/site-packages/ipykernel/kernelapp.py",

In [79]:
with tf.Session() as sess:
    for v in dataset2:
        print(v)
        _a=sess.run([v])
        print(_a)
    

Tensor("IteratorGetNext_421:0", shape=(?,), dtype=int32)


NotFoundError: Resource AnonymousIterator/AnonymousIterator11/N10tensorflow4data16IteratorResourceE does not exist.
	 [[node IteratorGetNext_421 (defined at <ipython-input-79-681e2babbb8d>:2) ]]

Original stack trace for 'IteratorGetNext_421':
  File "/Users/songfeng/APP/env/python/anaconda3/envs/tensorflow14/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/songfeng/APP/env/python/anaconda3/envs/tensorflow14/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/songfeng/APP/env/python/anaconda3/envs/tensorflow14/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/songfeng/APP/env/python/anaconda3/envs/tensorflow14/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/Users/songfeng/APP/env/python/anaconda3/envs/tensorflow14/lib/python3.7/site-packages/ipykernel/kernelapp.py",

In [59]:
dataset2 = tf.data.Dataset.from_tensor_slices([1, 2, 3])
dataset2 = dataset2.shuffle(buffer_size=10).batch(3)
iterator = dataset2.make_one_shot_iterator()
one_element = iterator.get_next()
print(one_element)
with tf.Session() as sess:
    for i in range(4):
        print(sess.run(one_element))

Tensor("IteratorGetNext_47:0", shape=(?,), dtype=int32)
[1 2 3]


OutOfRangeError: End of sequence
	 [[node IteratorGetNext_47 (defined at <ipython-input-59-20bea4c91511>:4) ]]

Errors may have originated from an input operation.
Input Source operations connected to node IteratorGetNext_47:
 OneShotIterator_18 (defined at <ipython-input-59-20bea4c91511>:3)

Original stack trace for 'IteratorGetNext_47':
  File "/Users/songfeng/APP/env/python/anaconda3/envs/tensorflow14/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/songfeng/APP/env/python/anaconda3/envs/tensorflow14/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/songfeng/APP/env/python/anaconda3/envs/tensorflow14/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/songfeng/APP/env/python/anaconda3/envs/tensorflow14/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/Users/songfen

In [84]:
dataset2 = tf.data.Dataset.from_tensor_slices([1, 2, 3,325,45,54654,456,546,7,4,223,4])
dataset2 = dataset2.shuffle(buffer_size=3).batch(3)
# iterator = dataset2.make_one_shot_iterator()
iterator = dataset2.make_initializable_iterator()
one_element = iterator.get_next()
print(one_element)
with tf.Session() as sess:
    sess.run(iterator.initializer)
    for i in range(10):
        print(v)
        print(sess.run(one_element))
#         print(sess.run(v))

Tensor("IteratorGetNext_426:0", shape=(?,), dtype=int32)
Tensor("IteratorGetNext_421:0", shape=(?,), dtype=int32)
[  1 325   2]
Tensor("IteratorGetNext_421:0", shape=(?,), dtype=int32)
[   45 54654   546]
Tensor("IteratorGetNext_421:0", shape=(?,), dtype=int32)
[456   3 223]
Tensor("IteratorGetNext_421:0", shape=(?,), dtype=int32)
[7 4 4]
Tensor("IteratorGetNext_421:0", shape=(?,), dtype=int32)


OutOfRangeError: End of sequence
	 [[node IteratorGetNext_426 (defined at <ipython-input-84-087e07cf7ccf>:5) ]]

Errors may have originated from an input operation.
Input Source operations connected to node IteratorGetNext_426:
 IteratorV2_6 (defined at <ipython-input-84-087e07cf7ccf>:4)

Original stack trace for 'IteratorGetNext_426':
  File "/Users/songfeng/APP/env/python/anaconda3/envs/tensorflow14/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/songfeng/APP/env/python/anaconda3/envs/tensorflow14/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/songfeng/APP/env/python/anaconda3/envs/tensorflow14/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/songfeng/APP/env/python/anaconda3/envs/tensorflow14/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/Users/songfeng/A